In [127]:
import pandas as pd
import datetime
import numpy as np


In [2]:

df = pd.read_csv('order_brush_order_new.csv')
base27=datetime.datetime.strptime('2019-12-27 00:00:00', '%Y-%m-%d %H:%M:%S')
df['seconds'] = df['event_time'].apply(lambda x : (datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')-base27).days*86400+(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')-base27).seconds)
df=df.sort_values(by='seconds')


In [3]:
df.describe(include='all')

,orderid,shopid,userid,event_time,seconds
count,2.227500e+05,2.227500e+05,2.227500e+05,222750,222750.000000
unique,NaN,NaN,NaN,121064,NaN
top,NaN,NaN,NaN,2019-12-30 23:31:45,NaN
freq,NaN,NaN,NaN,23,NaN
mean,3.130027e+13,9.433117e+07,9.802880e+07,NaN,225066.518290
std,1.222774e+11,5.695790e+07,6.839048e+07,NaN,122277.428744
min,3.107520e+13,1.000900e+04,1.000700e+04,NaN,0.000000
25%,3.120360e+13,4.980267e+07,3.508127e+07,NaN,128396.000000
50%,3.130561e+13,9.033636e+07,9.309625e+07,NaN,230414.000000
75%,3.140604e+13,1.475053e+08,1.590612e+08,NaN,330836.250000


In [4]:
df['event_time'].sort_values()

150060    2019-12-27 00:00:00
25985     2019-12-27 00:00:00
81020     2019-12-27 00:00:00
146598    2019-12-27 00:00:02
80355     2019-12-27 00:00:05
                 ...         
43965     2019-12-31 23:59:43
206585    2019-12-31 23:59:47
160657    2019-12-31 23:59:51
126212    2019-12-31 23:59:51
179881    2019-12-31 23:59:56
Name: event_time, Length: 222750, dtype: object

In [5]:
df["shopuser"] = df.shopid.astype(str) +'-'+ df.userid.astype(str)


In [6]:
df

,orderid,shopid,userid,event_time,seconds,shopuser
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00,0,6042309-97707522
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00,0,104804492-97707522
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00,0,8715449-97707522
146598,31075201870570,190969466,170182475,2019-12-27 00:00:02,2,190969466-170182475
80355,31075205798264,2859407,12532131,2019-12-27 00:00:05,5,2859407-12532131
...,...,...,...,...,...,...
114113,31507183252446,149254894,193333760,2019-12-31 23:59:43,431983,149254894-193333760
206585,31507187390691,147941492,40258063,2019-12-31 23:59:47,431987,147941492-40258063
126212,31507191066628,187123853,2338306,2019-12-31 23:59:51,431991,187123853-2338306
160657,31507191066627,154074176,2338306,2019-12-31 23:59:51,431991,154074176-2338306


In [142]:

#only analyze transactions that's more than 3.

g = df.groupby(['shopid','userid'])

sdf = g.filter(lambda x: len(x['orderid'])>=3)


shopids = sdf['shopid'].unique()
result_df = pd.DataFrame(columns=['shopid','userid'])

i=0

for si in shopids:
    i=i+1
    if i%100==0:
        print('i: ',i)

    tdf = sdf[sdf['shopid']==si]
    seconds = tdf['seconds'].unique()
#     seconds = [i*30 for i in range(0, 2*5760)] # 5760
#     print('seconds, ',seconds)
    # seconds = [0]

    
    # print('result:',result_df)

    for s in seconds:
#         if s%1000==0:
#             print('s:',s)

        hdf = tdf[(tdf['seconds'] >= s) & (tdf['seconds']<=s+3600)]
        a =hdf.groupby(['shopid'], as_index=False)['orderid'].count().rename(columns={'orderid':'count'}) # transaction count
        b = hdf.groupby(['shopid', 'userid'], as_index=False).size().reset_index().rename(columns={0:'count'})
    #     print('b:', b)
        b = b[['shopid', 'count']].groupby('shopid', as_index=False).count()

    #     d=hdf.groupby(['shopid', 'userid'], as_index=False).size()
    #     print('a:',a)

    #     print('b:',b)
    #     print(a)
    #     print(b)
        shopids =a[((a/b>=3)['count'])]['shopid']
    #     print('shopids:',shopids)

        for c in shopids:
            udf = hdf[hdf['shopid']==c]
    #         udf = df[df['shopid']==145777302]
            for u in udf['userid']:
                if sum(udf['userid']==u)>=3:
    #                 print(u)
                    rdf=pd.DataFrame({'shopid':[c], 'userid':[u]})
                    result_df = pd.concat([result_df,rdf])



i:  100
i:  200
i:  300
i:  400
i:  500
i:  600
i:  700
i:  800


In [143]:
result_df

,shopid,userid
0,91799978,214814254
0,91799978,214814254
0,91799978,214814254
0,91799978,15697974
0,91799978,15697974
...,...,...
0,96757644,215243653
0,96757644,215243653
0,189544563,799445
0,189544563,799445


In [144]:
suspicious_df = result_df[~result_df.duplicated()]
suspicious_df = suspicious_df.sort_values(by=['shopid','userid'])

In [145]:
suspicious_df.iloc[0]['shopid']

10159

In [146]:
data = pd.DataFrame(df['shopid'].unique(), columns = ['shopid'])
data['userid'] = '0'

for i in range(len(suspicious_df)):
    if data[data['shopid']==suspicious_df.iloc[i]['shopid']]['userid'].values=='0':
        data['userid'][data['shopid']==suspicious_df.iloc[i]['shopid']] = str(suspicious_df.iloc[i]['userid'])
#         print(str(suspicious_df.iloc[i]['userid']))
        print(data[data['shopid']==suspicious_df.iloc[i]['shopid']]['userid'])
        
    else:
        data['userid'][data['shopid']==suspicious_df.iloc[i]['shopid']] = data[data['shopid']==suspicious_df.iloc[i]['shopid']]['userid']+'&'+str(suspicious_df.iloc[i]['userid'])
        

data.to_csv('submission.csv', index=False)

<ipython-input-146-0bcb656006b1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['userid'][data['shopid']==suspicious_df.iloc[i]['shopid']] = str(suspicious_df.iloc[i]['userid'])
<ipython-input-146-0bcb656006b1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['userid'][data['shopid']==suspicious_df.iloc[i]['shopid']] = data[data['shopid']==suspicious_df.iloc[i]['shopid']]['userid']+'&'+str(suspicious_df.iloc[i]['userid'])


1102    214988798
Name: userid, dtype: object
443    198097381
Name: userid, dtype: object
107    77819
Name: userid, dtype: object
4986    672345
Name: userid, dtype: object
1934    214495220
Name: userid, dtype: object
3625    740844
Name: userid, dtype: object
4265    170385453
Name: userid, dtype: object
2160    190449497
Name: userid, dtype: object
923    214992524
Name: userid, dtype: object
846    212200633
Name: userid, dtype: object
4131    72914921
Name: userid, dtype: object
641    264511
Name: userid, dtype: object
6448    181682008
Name: userid, dtype: object
328    7670129
Name: userid, dtype: object
5878    75558350
Name: userid, dtype: object
3626    62618064
Name: userid, dtype: object
554    141006168
Name: userid, dtype: object
2713    565381
Name: userid, dtype: object
3636    188942105
Name: userid, dtype: object
1291    122277324
Name: userid, dtype: object
4028    181408876
Name: userid, dtype: object
3249    15053804
Name: userid, dtype: object
1574    78451300


5233    214226569
Name: userid, dtype: object
3152    188187242
Name: userid, dtype: object
4658    80682958
Name: userid, dtype: object
7521    214778616
Name: userid, dtype: object
1592    81928284
Name: userid, dtype: object
7940    213714377
Name: userid, dtype: object
1437    10051694
Name: userid, dtype: object
260    214431654
Name: userid, dtype: object
2231    148215831
Name: userid, dtype: object
1607    207428158
Name: userid, dtype: object
8960    1276208
Name: userid, dtype: object
9420    51759862
Name: userid, dtype: object
490    9753706
Name: userid, dtype: object
2679    35639374
Name: userid, dtype: object
3621    212200633
Name: userid, dtype: object
4909    212200633
Name: userid, dtype: object
7932    31233680
Name: userid, dtype: object
1952    214345977
Name: userid, dtype: object
870    157946285
Name: userid, dtype: object
1844    199703906
Name: userid, dtype: object
1635    133542654
Name: userid, dtype: object
1648    178640685
Name: userid, dtype: object
9

In [148]:
data[data['userid']!='0']


,shopid,userid
2,8715449,9753706
8,54615708,2973390&47919266&111115654&134786596&199416406
10,137739819,117286517
11,26218382,1233930
15,9374147,148176353
...,...,...
17530,203070501,189834273
17957,156995951,204919399
17981,129113839,215408773
18410,123401085,12869645
